# Dogs vs cats

In [25]:
import utils
import os
import h5py

import numpy as np

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical

In [2]:
PATH = 'data/'
RESULTS_DIR = PATH + 'results/'
SAVE_DIR = PATH + 'save/'

if not os.path.exists(RESULTS_DIR):
    os.mkdir(RESULTS_DIR)
if not os.path.exists(SAVE_DIR):
    os.mkdir(SAVE_DIR)

RESET_VALIDATION_SET = True
RELOAD_BATCHES = True
WITH_DATA_AUGMENTATION = False

img_shape = (3,224,224)

top_model_nb_epoch = 1
final_model_nb_epoch = 1

In [3]:
if RESET_VALIDATION_SET :
    split_valid = 0.1
    utils.move_validset_into_trainset(PATH, verbose=True)
    utils.generate_validation_set(PATH, val_split=split_valid, verbose=True)

1250	cat
1250	dog
1250/12500		cat
1250/12500		dog


In [4]:
train_size = 22500
valid_size = 2500

# VGG Model

In [5]:
vgg = Sequential()

vgg.add(ZeroPadding2D((1, 1), input_shape=(3,224,224)))
vgg.add(Convolution2D(64, 3, 3, activation='relu'))
vgg.add(ZeroPadding2D((1, 1)))
vgg.add(Convolution2D(64, 3, 3, activation='relu'))
vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

vgg.add(ZeroPadding2D((1, 1)))
vgg.add(Convolution2D(128, 3, 3, activation='relu'))
vgg.add(ZeroPadding2D((1, 1)))
vgg.add(Convolution2D(128, 3, 3, activation='relu'))
vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

vgg.add(ZeroPadding2D((1, 1)))
vgg.add(Convolution2D(256, 3, 3, activation='relu'))
vgg.add(ZeroPadding2D((1, 1)))
vgg.add(Convolution2D(256, 3, 3, activation='relu'))
vgg.add(ZeroPadding2D((1, 1)))
vgg.add(Convolution2D(256, 3, 3, activation='relu'))
vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

vgg.add(ZeroPadding2D((1, 1)))
vgg.add(Convolution2D(512, 3, 3, activation='relu'))
vgg.add(ZeroPadding2D((1, 1)))
vgg.add(Convolution2D(512, 3, 3, activation='relu'))
vgg.add(ZeroPadding2D((1, 1)))
vgg.add(Convolution2D(512, 3, 3, activation='relu'))
vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

vgg.add(ZeroPadding2D((1, 1)))
vgg.add(Convolution2D(512, 3, 3, activation='relu'))
vgg.add(ZeroPadding2D((1, 1)))
vgg.add(Convolution2D(512, 3, 3, activation='relu'))
vgg.add(ZeroPadding2D((1, 1)))
vgg.add(Convolution2D(512, 3, 3, activation='relu'))
vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

vgg.add(Flatten())
vgg.add(Dense(4096, activation='relu'))
vgg.add(BatchNormalization())
vgg.add(Dropout(0.5))
vgg.add(Dense(4096, activation='relu'))
vgg.add(BatchNormalization())
vgg.add(Dropout(0.5))
vgg.add(Dense(1000, activation='softmax'))

Load VGG16 weights
http://www.platform.ai/models/vgg16_bn.h5

In [6]:
vgg.load_weights('data/model/vgg16_bn.h5')

In [7]:
for i in range(8):
    vgg.pop()

In [8]:
vgg_train_generator = ImageDataGenerator().flow_from_directory(
    PATH + 'train',
    target_size=(img_shape[1], img_shape[2]),
    batch_size=10,
    class_mode=None,
    shuffle=False
)

Found 22500 images belonging to 2 classes.


In [9]:
vgg_train_predictions = vgg.predict_generator(
    vgg_train_generator,
    #vgg_train_generator.nb_sample
    train_size
)
np.save(
    SAVE_DIR + 'vgg_train_predictions.npy',
    vgg_train_predictions
)

In [10]:
vgg_train_predictions = np.load(SAVE_DIR + 'vgg_train_predictions.npy')

In [11]:
vgg_train_labels = np.array([0] * (train_size/2) + [1] * (train_size/2))

In [12]:
vgg_valid_generator = ImageDataGenerator().flow_from_directory(
    PATH + 'valid',
    target_size=(img_shape[1], img_shape[2]),
    batch_size=10,
    class_mode=None,
    shuffle=False
)

Found 2500 images belonging to 2 classes.


In [13]:
vgg_valid_predictions = vgg.predict_generator(
    vgg_valid_generator,
    #vgg_valid_generator.nb_sample
    valid_size
)
np.save(
    SAVE_DIR + 'vgg_valid_predictions.npy',
    vgg_valid_predictions
)

In [14]:
vgg_valid_predictions = np.load(SAVE_DIR + 'vgg_valid_predictions.npy')

In [15]:
vgg_valid_labels = np.array([0] * (valid_size/2) + [1] * (valid_size/2))

In [16]:
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg_train_predictions.shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [17]:
top_model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [20]:
top_model.fit(
    vgg_train_predictions,
    vgg_train_labels,
    nb_epoch=50,
    batch_size=10,
    validation_data=(vgg_valid_predictions, vgg_valid_labels))

Train on 22500 samples, validate on 2500 samples
Epoch 1/50
22500/22500 [==============================] - 37s - loss: 0.9671 - acc: 0.9379 - val_loss: 0.6632 - val_acc: 0.9576
Epoch 2/50
22500/22500 [==============================] - 38s - loss: 0.8361 - acc: 0.9468 - val_loss: 0.6066 - val_acc: 0.9608
Epoch 3/50
22500/22500 [==============================] - 40s - loss: 0.7737 - acc: 0.9510 - val_loss: 1.3601 - val_acc: 0.9140
Epoch 4/50
22500/22500 [==============================] - 39s - loss: 0.7936 - acc: 0.9495 - val_loss: 0.6163 - val_acc: 0.9608
Epoch 5/50
22500/22500 [==============================] - 40s - loss: 0.7091 - acc: 0.9551 - val_loss: 0.5602 - val_acc: 0.9644
Epoch 6/50
22500/22500 [==============================] - 41s - loss: 0.7253 - acc: 0.9540 - val_loss: 0.5225 - val_acc: 0.9672
Epoch 7/50
22500/22500 [==============================] - 40s - loss: 0.7315 - acc: 0.9541 - val_loss: 0.5789 - val_acc: 0.9636
Epoch 8/50
22500/22500 [==============================]

In [21]:
top_model.save_weights(SAVE_DIR + 'vgg_topvgg.h5')

In [22]:
vgg.add(top_model)

In [23]:
for layer in vgg.layers[:25]:
    layer.trainable = False

In [37]:
vgg.compile(
    loss='binary_crossentropy',
    optimizer=SGD(lr=0.0001, momentum=0.9),
    metrics=['accuracy']
)

In [38]:
final_train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    zoom_range=0.2,
    height_shift_range=0.05,
    shear_range=0.1,
    horizontal_flip=True
)

In [39]:
final_train_generator = final_train_datagen.flow_from_directory(
    PATH + 'train',
    target_size=(img_shape[1], img_shape[2]),
    batch_size=10,
    class_mode='binary'
)

Found 22500 images belonging to 2 classes.


In [40]:
final_valid_generator = ImageDataGenerator().flow_from_directory(
    PATH + 'valid',
    target_size=(img_shape[1], img_shape[2]),
    batch_size=10,
    class_mode='binary'
)

Found 2500 images belonging to 2 classes.


In [41]:
vgg.fit_generator(
    final_train_generator,
    train_size,
    nb_epoch=final_model_nb_epoch,
    validation_data=final_valid_generator,
    nb_val_samples=valid_size
)

Epoch 1/1
 2960/22500 [==>...........................] - ETA: 1298s - loss: 7.9719 - acc: 0.5054

KeyboardInterrupt: 

In [ ]:
test_generator = ImageDataGenerator().flow_from_directory(
    PATH + 'test',
    target_size=((img_shape[1], img_shape[2])),
    batch_size=10,
    class_mode=None,
    shuffle=False
)

In [ ]:
vgg.predict_generator(
    test_generator,
    test_generator.nb_sample
)

# Save predictions to CSV for Kaggle submission

In [ ]:
filenames = [int((f[len('unknown/'):])[:-len('.jpg')]) for f in batches_test.filenames]

In [ ]:
isdog = np.array([float(i[1]) for i in vgg_preds_test])
isdog = isdog.clip(min=0.05, max=0.95)

In [ ]:
np.savetxt(
    RESULTS_DIR + 'submission.csv',
    np.stack([filenames,isdog], axis=1),
    fmt='%d,%.5f',
    header='id,label',
    comments=''
)

In [ ]:
from IPython.display import FileLink
FileLink(RESULTS_DIR + 'submission.csv')

# Predictions examples

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image

i = np.random.randint(len(isdog))
print 'is dog : {}'.format(isdog[i])
img = Image.open('data/test/unknown/{}.jpg'.format(filenames[i]))
plt.imshow(img)
plt.show()